# Data Preprocessing

The next step is to

In [1]:
#@title Imports for Project
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../data/rideshare_kaggle.csv')
df.head()

,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,9,16,12,2018-12-16 09:30:07,America/New_York,Haymarket Square,North Station,Lyft,...,0.1276,1544979600,39.89,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,2,27,11,2018-11-27 02:00:23,America/New_York,Haymarket Square,North Station,Lyft,...,0.1300,1543251600,40.49,1543233600,47.30,1543251600,36.20,1543291200,43.92,1543251600
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,1,28,11,2018-11-28 01:00:22,America/New_York,Haymarket Square,North Station,Lyft,...,0.1064,1543338000,35.36,1543377600,47.55,1543320000,31.04,1543377600,44.12,1543320000
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,4,30,11,2018-11-30 04:53:02,America/New_York,Haymarket Square,North Station,Lyft,...,0.0000,1543507200,34.67,1543550400,45.03,1543510800,30.30,1543550400,38.53,1543510800
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,3,29,11,2018-11-29 03:49:20,America/New_York,Haymarket Square,North Station,Lyft,...,0.0001,1543420800,33.10,1543402800,42.18,1543420800,29.11,1543392000,35.75,1543420800


Data Cleaning

In [3]:
#Identify number of null values per column in the database
df.isna().sum()

id                                 0
timestamp                          0
hour                               0
day                                0
month                              0
datetime                           0
timezone                           0
source                             0
destination                        0
cab_type                           0
product_id                         0
name                               0
price                          55095
distance                           0
surge_multiplier                   0
latitude                           0
longitude                          0
temperature                        0
apparentTemperature                0
short_summary                      0
long_summary                       0
precipIntensity                    0
precipProbability                  0
humidity                           0
windSpeed                          0
windGust                           0
windGustTime                       0
v

In [4]:
#Drop rows that have a null value in the price column
df = df[df['price'].notna()]
df.isna().sum()

id                             0
timestamp                      0
hour                           0
day                            0
month                          0
datetime                       0
timezone                       0
source                         0
destination                    0
cab_type                       0
product_id                     0
name                           0
price                          0
distance                       0
surge_multiplier               0
latitude                       0
longitude                      0
temperature                    0
apparentTemperature            0
short_summary                  0
long_summary                   0
precipIntensity                0
precipProbability              0
humidity                       0
windSpeed                      0
windGust                       0
windGustTime                   0
visibility                     0
temperatureHigh                0
temperatureHighTime            0
temperatur

In [5]:
#Remove duplicates in the ID column
df = df.drop_duplicates('id')

In [6]:
# Check for duplicate rows
duplicates = df.duplicated()
duplicates_num = duplicates.sum()

print(duplicates_num)

0


In [7]:
# Reduce data to only contain longitude between -180 and 180, and latitude between -90 and 90
df = df[(df['longitude'] > -180) & (df['longitude'] < 180)]
df = df[(df['latitude'] > -90) & (df['latitude'] < 90)]

In [8]:
# Drop unnecessary Lyft data
df = df[df['cab_type'] != 'Lyft']

df['cab_type']

12        Uber
13        Uber
14        Uber
15        Uber
16        Uber
          ... 
693065    Uber
693066    Uber
693067    Uber
693069    Uber
693070    Uber
Name: cab_type, Length: 330568, dtype: object

In [9]:
# Remove duplicate column

(df['visibility'] == df['visibility.1']).all()
df.drop('visibility.1', axis=1, inplace=True)

In [12]:
# Removing outliers in the numerical columns

columns_to_check = ['price', 'distance', 'surge_multiplier', 'temperature', 'apparentTemperature', 'precipIntensity', 'precipProbability', 'humidity', 'windSpeed', 'windGust', 'visibility', 'temperatureHigh', 'temperatureLow', 'apparentTemperatureHigh', 'apparentTemperatureLow', 'dewPoint', 'pressure', 'uvIndex', 'ozone', 'moonPhase', 'precipIntensityMax', 'temperatureMin', 'temperatureMax', 'apparentTemperatureMin', 'apparentTemperatureMax']

outliers = {}
for column in columns_to_check:
    mean = df[column].mean()
    std = df[column].std()
    outliers_mask = (df[column] < mean - 3 * std) | (df[column] > mean + 3 * std)
    outliers[column] = df[column][outliers_mask]

for column, outliers_data in outliers.items():
    if not outliers_data.empty:
        print(f"Outliers in column '{column}':")
        print(outliers_data)
        print("\n")

Outliers in column 'price':
2104      42.0
2143      46.5
2219      50.5
2567      41.5
2686      56.0
          ... 
692103    49.5
692205    42.0
692439    47.0
692962    51.0
693007    49.5
Name: price, Length: 2380, dtype: float64


Outliers in column 'distance':
516       7.46
2214      7.34
2215      7.34
2216      7.34
2218      7.34
          ... 
693005    7.36
693006    7.36
693007    7.36
693008    7.36
693009    7.36
Name: distance, Length: 3192, dtype: float64


Outliers in column 'temperature':
61        18.97
116       19.28
566       18.97
686       18.97
817       18.97
          ...  
690952    19.28
692920    18.97
692921    18.97
692922    18.97
692923    18.97
Name: temperature, Length: 2576, dtype: float64


Outliers in column 'precipIntensity':
42        0.1299
68        0.1299
204       0.0920
238       0.1267
241       0.1088
           ...  
692738    0.1264
692739    0.1264
692804    0.1289
692805    0.1289
692806    0.1289
Name: precipIntensity, Length: 1314

In [14]:
# Separate the Datetime column into time, year, month, date and day of the week

df['datetime'] = pd.to_datetime(df['datetime'])

df['time'] = df['datetime'].dt.strftime('%H:%M')
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.strftime('%B')
df['day'] = df['datetime'].dt.day
df['day_of_week'] = df['datetime'].dt.dayofweek 

day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df['day_of_week'] = df['day_of_week'].map(day_names)

# Reorder columns to ensure time/ date columns are together
df = df[['id', 'timestamp', 'day_of_week', 'month', 'day', 'year', 'time', 'datetime', 'hour',  'timezone', 'source', 'destination', 'cab_type', 'product_id', 'name', 'price', 'distance', 'surge_multiplier', 'latitude', 'longitude', 'temperature', 'apparentTemperature', 'short_summary', 'long_summary', 'precipIntensity', 'precipProbability', 'humidity', 'windSpeed', 'windGust', 'windGustTime', 'visibility', 'temperatureHigh', 'temperatureHighTime', 'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime', 'apparentTemperatureLow', 'apparentTemperatureLowTime', 'icon', 'dewPoint', 'pressure', 'windBearing', 'cloudCover', 'uvIndex', 'ozone', 'sunriseTime', 'sunsetTime', 'moonPhase', 'precipIntensityMax', 'uvIndexTime', 'temperatureMin', 'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin', 'apparentTemperatureMinTime', 'apparentTemperatureMax', 'apparentTemperatureMaxTime']]
df.head()

,id,timestamp,day_of_week,month,day,year,time,datetime,hour,timezone,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
12,009e9c53-074d-43cf-aef2-0fbc7a47ed3d,1.543616e+09,Friday,November,30,2018,22:13,2018-11-30 22:13:01,22,America/New_York,...,0.0003,1543593600,28.79,1543579200,42.52,1543600800,26.41,1543575600,40.53,1543611600
13,23f145da-f0c1-4d1f-a184-496bc003a7db,1.544698e+09,Thursday,December,13,2018,10:50,2018-12-13 10:50:11,10,America/New_York,...,0.0001,1544716800,18.29,1544688000,33.83,1544731200,13.79,1544688000,32.85,1544734800
14,357559cb-8c58-4278-a41a-e33b2e0997a3,1.544729e+09,Thursday,December,13,2018,19:15,2018-12-13 19:15:03,19,America/New_York,...,0.0001,1544716800,18.29,1544688000,33.83,1544731200,13.79,1544688000,32.85,1544734800
15,50ef1165-9d23-416c-a65c-18906207b295,1.545005e+09,Sunday,December,16,2018,23:55,2018-12-16 23:55:11,23,America/New_York,...,0.1252,1544979600,39.22,1544954400,43.83,1544990400,33.98,1545019200,38.38,1544986800
16,91c4861c-1780-42b0-bca1-bbd64a422cc3,1.544748e+09,Friday,December,14,2018,00:40,2018-12-14 00:40:07,0,America/New_York,...,0.0001,1544716800,18.29,1544688000,33.83,1544731200,13.79,1544688000,32.85,1544734800


In [ ]:
# Save cleaned dataframe
df.to_csv('cleaned_data.csv', index=False)
